In [1]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.svm import SVC
import db_settings
import pyexasol
from pathlib import Path
import warnings
import csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
print(os.getcwd())
np.random.seed(1234)

C:\Users\janja\Desktop\GitHub\lidar-vegetation-data


In [2]:
def get_class(id):
    family = dict_families[dict_gattung[id]['ID_FAMILIE']]
    order = dict_order[family['ID_ORDNUNG']]
    c = order['ID_KLASSE']
    return c
def get_order(id):
    family = dict_families[dict_gattung[id]['ID_FAMILIE']]
    o = family['ID_ORDNUNG']
    return o
def get_family(id):
    f = dict_families[dict_gattung[id]['ID_FAMILIE']]
    return f


def scale_y(y):
    """translates a list of labels with unsorted numbers to a labeling starting with [0,1,2,...]

    Args:
        y (_type_): _description_

    Returns:
        _type_: _description_
    """
    labels = np.unique(y)
    y_new = [i for i in range(len(labels))]
    translator = {key:value for (key,value) in zip(labels, y_new)}
    re_translate = {key:value for (key,value) in zip(y_new, labels)}

    y = [translator[i] for i in y]
    return y, re_translate
        
         

def train(X_train, X_test, y_train, y_test, weight=True):
    if weight==True:
        w = 'balanced'
    else:
        w=None
    print("SVM linear Kernel:")
    svc_linear = SVC(kernel='linear', probability=True, class_weight=w)  #, class_weight='balanced'
    svc_linear.fit(X_train, y_train)
    y_pred = svc_linear.predict(X_test)
    print("accuracy_score: ", accuracy_score(y_test, y_pred))
    print("balanced accuracy score", balanced_accuracy_score(y_test, y_pred, adjusted=True))
    print("SVM rbf kernel:")
    svc_rbf = SVC(kernel='rbf', probability=True, class_weight=w)
    svc_rbf.fit(X_train, y_train)
    y_pred = svc_rbf.predict(X_test)
    print("accuracy_score: ", accuracy_score(y_test, y_pred))
    print("balanced accuracy score", balanced_accuracy_score(y_test, y_pred, adjusted=True))
    print("SVM poly kernel:")
    svc_poly = SVC(kernel='poly', probability=True, class_weight=w)
    svc_poly.fit(X_train, y_train)
    y_pred = svc_poly.predict(X_test)
    print("accuracy_score: ", accuracy_score(y_test, y_pred))
    print("balanced accuracy score", balanced_accuracy_score(y_test, y_pred, adjusted=True))
    print("decission Tree")
    t = tree.DecisionTreeClassifier(class_weight=w)
    t = t.fit(X_train, y_train)
    print("accuracy_score: ", t.score(X_test, y_test))
    print("balanced accuracy score", balanced_accuracy_score(y_test, t.predict(X_test), adjusted=True))
    print("decission tree depth=", 20)
    decision_tree = DecisionTreeClassifier(random_state=0, max_depth=29, class_weight=w)
    decision_tree = decision_tree.fit(X_train, y_train)
    print("accuracy_score: ", decision_tree.score(X_test, y_test))
    print("balanced accuracy score", balanced_accuracy_score(y_test, decision_tree.predict(X_test), adjusted=True))
    #random forest:
    print("random forest:")
    forest = RandomForestClassifier(n_estimators=500, class_weight=w)
    forest.fit(X_train,y_train)
    forest.predict(X_test)
    forest.predict_proba(X_test)
    print("accuracy_score: ", forest.score(X_test, y_test))
    print("balanced accuracy score", balanced_accuracy_score(y_test, forest.predict(X_test), adjusted=True))

def get_data():
    df = pd.read_csv('feature_list_color.csv')
    labels_unedited = np.array(df['ID_GATTUNG'])
    df =df.drop(columns=['ID_GATTUNG'])
    features = []  
    # Iterate over each row
    for rows in df.itertuples():      
        # append the list to the final list
        features.append(list(rows[1:]))
    return features, labels_unedited
    

    

In [3]:
db =db_settings.db(autocommit=False)


In [4]:
req_families = """SELECT * FROM lidar_proj.familien"""
req_order = """SELECT * FROM lidar_proj.ordnungen"""
req_class = """SELECT * FROM lidar_proj.klassen"""
req_gattung = """SELECT * FROM lidar_proj.gattungen"""

df_families = db.export_to_pandas(req_families)
df_order = db.export_to_pandas(req_order)
df_class = db.export_to_pandas(req_class)
df_gattung = db.export_to_pandas(req_gattung)

df_families.set_index("ID", drop=True, inplace=True)
df_order.set_index("ID", drop=True, inplace=True)
df_class.set_index("ID", drop=True, inplace=True)
df_gattung.set_index("ID", drop=True, inplace=True)

In [5]:
global dict_gattung
global dict_families
global dict_order
global dict_class

dict_gattung =df_gattung.to_dict(orient="index")

dict_families =df_families.to_dict(orient="index")
dict_order =df_order.to_dict(orient="index")
dict_class =df_class.to_dict(orient="index")
print(dict_gattung[1])


{'LAT_NAME': 'Abies', 'DT_NAME': ' \tTannen ', 'ID_FAMILIE': 1}


In [6]:
#luckup klasse
gattungs_id = 3
print(dict_gattung[gattungs_id])
family = dict_families[dict_gattung[gattungs_id]['ID_FAMILIE']]
print(family)
order = dict_order[family['ID_ORDNUNG']]
print(order)
c = dict_class[order['ID_KLASSE']]
print(c)


{'LAT_NAME': 'Aesculus', 'DT_NAME': ' \tRosskastanien  ', 'ID_FAMILIE': 2}
{'LAT_NAME': 'Sapindaceae', 'DT_NAME': ' Seifenbaumgewächse ', 'ID_ORDNUNG': 4}
{'LAT_NAME': 'Sapindales', 'DT_NAME': 'Seifenbaumartige ', 'ID_KLASSE': 1}
{'LAT_NAME': 'Magnoliopsida', 'DT_NAME': ' Bedecktsamer'}


In [7]:
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order = (14 ,'Gingko')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)

{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Gingko
labels train set {0.0: 113, 1.0: 112}
labels test set {0.0: 37, 1.0: 38}
SVM linear Kernel:
accuracy_score:  0.6266666666666667
balanced accuracy score 0.2567567567567568
SVM rbf kernel:
accuracy_score:  0.49333333333333335
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.49333333333333335
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6133333333333333
balanced accuracy score 0.22688477951635844
decission tree depth= 20
accuracy_score:  0.56
balanced accuracy score 0.11948790896159323
random forest:
accuracy_score:  0.7733333333333333
balanced accuracy score 0.5476529160739687


In [8]:
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order = (7, 'Fagales')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)

{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Fagales
labels train set {0.0: 279, 1.0: 259}
labels test set {0.0: 80, 1.0: 100}
SVM linear Kernel:
accuracy_score:  0.5333333333333333
balanced accuracy score 0.09250000000000003
SVM rbf kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.55
balanced accuracy score 0.11499999999999999
decission tree depth= 20
accuracy_score:  0.5111111111111111
balanced accuracy score 0.03750000000000009
random forest:
accuracy_score:  0.6722222222222223
balanced accuracy score 0.355


In [9]:
"""Downsamplig Magniolopsida, droping gingko:"""

features, labels_unedited =get_data()
labels = np.array(list(map(get_class, labels_unedited)))
#labels = np.array(list(map(lambda x: 2 if x==3 else x, labels)))

#mappimg ginko as conifera
print(len(labels), " individuums in train/test-set")

labels, re_translator =scale_y(labels)
#count_labels 
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

labels = np.array(labels)
features = np.array(features)
#downsamplig:
features_0 = features[ labels == 0]
features_1 = features[ labels == 1]
features_2 = features[ labels == 2]

#features_0_downsampled = np.random.choice(features_0, size=features_1.shape[0], replace=False)
idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
features_0_downsampled =features_0[idx,:]

labels_0 = np.zeros(len(features_0_downsampled))
labels_1 = np.ones(len(features_1))

features =np.concatenate((features_0_downsampled, features_1))
labels = np.concatenate((labels_0, labels_1))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

print(features.shape)
print(labels.shape)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)


33733  individuums in train/test-set
{0: 33224, 1: 359, 2: 150}
(718, 160)
(718,)
labels train set {0.0: 259, 1.0: 279}
labels test set {0.0: 100, 1.0: 80}
SVM linear Kernel:
accuracy_score:  0.5166666666666667
balanced accuracy score 0.030000000000000027
SVM rbf kernel:
accuracy_score:  0.45
balanced accuracy score 0.010000000000000009
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6
balanced accuracy score 0.20250000000000012
decission tree depth= 20
accuracy_score:  0.6
balanced accuracy score 0.20250000000000012
random forest:
accuracy_score:  0.6833333333333333
balanced accuracy score 0.39250000000000007


In [8]:
"""ordungen: 3: Coniferales=371
  6 Fabales= 2465
24 Males: 8782
23 Proteales: 3100
16 Malphigiales: 456
"""
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order = (8 ,'Rosales')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)



    
"""
{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}
comparing coniferales (Nadelhölzer) to  Rosales
labels train set {0.0: 289, 1.0: 269}
labels test set {0.0: 83, 1.0: 103}
SVM linear Kernel:
accuracy_score:  0.46774193548387094
balanced accuracy score -0.0641010644519826
SVM rbf kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.7096774193548387
balanced accuracy score 0.43361796701368593
decission tree depth= 20
accuracy_score:  0.6666666666666666
balanced accuracy score 0.34659024447303777
random forest:
accuracy_score:  0.7311827956989247
balanced accuracy score 0.4701134635629898

"""    





 

{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Rosales
labels train set {0.0: 279, 1.0: 259}
labels test set {0.0: 80, 1.0: 100}
SVM linear Kernel:
accuracy_score:  0.6555555555555556
balanced accuracy score 0.32499999999999996
SVM rbf kernel:
accuracy_score:  0.45555555555555555
balanced accuracy score 0.020000000000000018
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6222222222222222
balanced accuracy score 0.23750000000000004
decission tree depth= 20
accuracy_score:  0.6166666666666667
balanced accuracy score 0.22750000000000004
random forest:
accuracy_score:  0.7388888888888889
balanced accuracy score 0.49


'\n{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}\ncomparing coniferales (Nadelhölzer) to  Rosales\nlabels train set {0.0: 289, 1.0: 269}\nlabels test set {0.0: 83, 1.0: 103}\nSVM linear Kernel:\naccuracy_score:  0.46774193548387094\nbalanced accuracy score -0.0641010644519826\nSVM rbf kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\nSVM poly kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\ndecission Tree\naccuracy_score:  0.7096774193548387\nbalanced accuracy score 0.43361796701368593\ndecission tree depth= 20\naccuracy_score:  0.6666666666666666\nbalanced accuracy score 0.34659024447303777\nrandom forest:\naccuracy_score:  0.7311827956989247\nbalanced accuracy score 0.4701134635629898\n\n'

In [9]:

print(" ")
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order =(10, 'Lamiales')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)

"""
{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}
comparing coniferales (Nadelhölzer) to  Lamiales
labels train set {0.0: 289, 1.0: 269}
labels test set {0.0: 83, 1.0: 103}
SVM linear Kernel:
accuracy_score:  0.553763440860215
balanced accuracy score 0.13100947479237335
SVM rbf kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.44086021505376344
balanced accuracy score -0.012048192771084376
decission Tree
accuracy_score:  0.6666666666666666
balanced accuracy score 0.3325535150309977
decission tree depth= 20
accuracy_score:  0.6827956989247311
balanced accuracy score 0.36635863843724414
random forest:
accuracy_score:  0.6881720430107527
balanced accuracy score 0.39712247046438165
"""

    

 
{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Lamiales
labels train set {0.0: 279, 1.0: 259}
labels test set {0.0: 80, 1.0: 100}
SVM linear Kernel:
accuracy_score:  0.6277777777777778
balanced accuracy score 0.26
SVM rbf kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6
balanced accuracy score 0.19499999999999984
decission tree depth= 20
accuracy_score:  0.5833333333333334
balanced accuracy score 0.1625000000000001
random forest:
accuracy_score:  0.7055555555555556
balanced accuracy score 0.4224999999999999


'\n{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}\ncomparing coniferales (Nadelhölzer) to  Lamiales\nlabels train set {0.0: 289, 1.0: 269}\nlabels test set {0.0: 83, 1.0: 103}\nSVM linear Kernel:\naccuracy_score:  0.553763440860215\nbalanced accuracy score 0.13100947479237335\nSVM rbf kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\nSVM poly kernel:\naccuracy_score:  0.44086021505376344\nbalanced accuracy score -0.012048192771084376\ndecission Tree\naccuracy_score:  0.6666666666666666\nbalanced accuracy score 0.3325535150309977\ndecission tree depth= 20\naccuracy_score:  0.6827956989247311\nbalanced accuracy score 0.36635863843724414\nrandom forest:\naccuracy_score:  0.6881720430107527\nbalanced accuracy score 0.39712247046438165\n'

In [10]:
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order = (6, 'Fabales')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)

"""
{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}
comparing coniferales (Nadelhölzer) to  Fabales
labels train set {0.0: 289, 1.0: 269}
labels test set {0.0: 83, 1.0: 103}
SVM linear Kernel:
accuracy_score:  0.6129032258064516
balanced accuracy score 0.2424845011112411
SVM rbf kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6505376344086021
balanced accuracy score 0.29874839162475153
decission tree depth= 20
accuracy_score:  0.6344086021505376
balanced accuracy score 0.2719616329395249
random forest:
accuracy_score:  0.7311827956989247
balanced accuracy score 0.4794712831910166

"""

    

{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Fabales
labels train set {0.0: 279, 1.0: 259}
labels test set {0.0: 80, 1.0: 100}
SVM linear Kernel:
accuracy_score:  0.6
balanced accuracy score 0.2124999999999999
SVM rbf kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6111111111111112
balanced accuracy score 0.23249999999999993
decission tree depth= 20
accuracy_score:  0.6222222222222222
balanced accuracy score 0.25249999999999995
random forest:
accuracy_score:  0.7222222222222222
balanced accuracy score 0.45500000000000007


'\n{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}\ncomparing coniferales (Nadelhölzer) to  Fabales\nlabels train set {0.0: 289, 1.0: 269}\nlabels test set {0.0: 83, 1.0: 103}\nSVM linear Kernel:\naccuracy_score:  0.6129032258064516\nbalanced accuracy score 0.2424845011112411\nSVM rbf kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\nSVM poly kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\ndecission Tree\naccuracy_score:  0.6505376344086021\nbalanced accuracy score 0.29874839162475153\ndecission tree depth= 20\naccuracy_score:  0.6344086021505376\nbalanced accuracy score 0.2719616329395249\nrandom forest:\naccuracy_score:  0.7311827956989247\nbalanced accuracy score 0.4794712831910166\n\n'

In [11]:
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order = (24, 'Malves')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)

"""
{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}
comparing coniferales (Nadelhölzer) to  Malves
labels train set {0.0: 289, 1.0: 269}
labels test set {0.0: 83, 1.0: 103}
SVM linear Kernel:
accuracy_score:  0.6989247311827957
balanced accuracy score 0.4142004912855306
SVM rbf kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6505376344086021
balanced accuracy score 0.2870511170897181
decission tree depth= 20
accuracy_score:  0.6182795698924731
balanced accuracy score 0.2287986899052521
random forest:
accuracy_score:  0.7526881720430108
balanced accuracy score 0.5323429640893673

"""

    

{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Malves
labels train set {0.0: 279, 1.0: 259}
labels test set {0.0: 80, 1.0: 100}
SVM linear Kernel:
accuracy_score:  0.6111111111111112
balanced accuracy score 0.22750000000000004
SVM rbf kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.5388888888888889
balanced accuracy score 0.08749999999999991
decission tree depth= 20
accuracy_score:  0.5722222222222222
balanced accuracy score 0.1499999999999999
random forest:
accuracy_score:  0.6944444444444444
balanced accuracy score 0.41500000000000004


'\n{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}\ncomparing coniferales (Nadelhölzer) to  Malves\nlabels train set {0.0: 289, 1.0: 269}\nlabels test set {0.0: 83, 1.0: 103}\nSVM linear Kernel:\naccuracy_score:  0.6989247311827957\nbalanced accuracy score 0.4142004912855306\nSVM rbf kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\nSVM poly kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\ndecission Tree\naccuracy_score:  0.6505376344086021\nbalanced accuracy score 0.2870511170897181\ndecission tree depth= 20\naccuracy_score:  0.6182795698924731\nbalanced accuracy score 0.2287986899052521\nrandom forest:\naccuracy_score:  0.7526881720430108\nbalanced accuracy score 0.5323429640893673\n\n'

In [12]:
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order = (23, 'Proteales')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)

"""
{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}
comparing coniferales (Nadelhölzer) to  Proteales
labels train set {0.0: 289, 1.0: 269}
labels test set {0.0: 83, 1.0: 103}
SVM linear Kernel:
accuracy_score:  0.532258064516129
balanced accuracy score 0.08749561352204926
SVM rbf kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6182795698924731
balanced accuracy score 0.23815650953327872
decission tree depth= 20
accuracy_score:  0.6236559139784946
balanced accuracy score 0.24318633758334318
random forest:
accuracy_score:  0.6935483870967742
balanced accuracy score 0.4021522985144461

"""

    

{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Proteales
labels train set {0.0: 279, 1.0: 259}
labels test set {0.0: 80, 1.0: 100}
SVM linear Kernel:
accuracy_score:  0.5166666666666667
balanced accuracy score 0.03750000000000009
SVM rbf kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.5611111111111111
balanced accuracy score 0.14250000000000007
decission tree depth= 20
accuracy_score:  0.5388888888888889
balanced accuracy score 0.08999999999999986
random forest:
accuracy_score:  0.6722222222222223
balanced accuracy score 0.3624999999999998


'\n{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}\ncomparing coniferales (Nadelhölzer) to  Proteales\nlabels train set {0.0: 289, 1.0: 269}\nlabels test set {0.0: 83, 1.0: 103}\nSVM linear Kernel:\naccuracy_score:  0.532258064516129\nbalanced accuracy score 0.08749561352204926\nSVM rbf kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\nSVM poly kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\ndecission Tree\naccuracy_score:  0.6182795698924731\nbalanced accuracy score 0.23815650953327872\ndecission tree depth= 20\naccuracy_score:  0.6236559139784946\nbalanced accuracy score 0.24318633758334318\nrandom forest:\naccuracy_score:  0.6935483870967742\nbalanced accuracy score 0.4021522985144461\n\n'

In [13]:
features, labels_unedited =get_data()
labels = np.array(list(map(get_order, labels_unedited)))
features= np.array(features)
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

#(14, 'Gingkoales'), 
order =(16, 'Malphgiales')
print("comparing coniferales (Nadelhölzer) to ", order[1])
order_id = order[0]

features_0 = features[ labels == 3]
features_1 = features[ labels == order_id]

if features_0.shape[0] >  features_1.shape[0]:
    idx = np.random.randint(features_0.shape[0] -1, size=features_1.shape[0])
    features_0_downsampled =features_0[idx,:]        
    labels_0 = np.zeros(len(features_0_downsampled))
    labels_1 = np.ones(len(features_1))

    features =np.concatenate((features_0_downsampled, features_1))
    labels = np.concatenate((labels_0, labels_1))

else:
    idx = np.random.randint(features_1.shape[0] -1, size=features_0.shape[0])
    features_1_downsampled =features_1[idx,:]

    labels_1 = np.ones(len(features_1_downsampled))
    labels_0 = np.zeros(len(features_0))

    features =np.concatenate((features_1_downsampled, features_0))
    labels = np.concatenate((labels_1, labels_0))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test, weight=False)

"""
{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}
comparing coniferales (Nadelhölzer) to  Malphgiales
labels train set {0.0: 289, 1.0: 269}
labels test set {0.0: 83, 1.0: 103}
SVM linear Kernel:
accuracy_score:  0.5483870967741935
balanced accuracy score 0.10258509767224222
SVM rbf kernel:
accuracy_score:  0.46236559139784944
balanced accuracy score 0.026786758685226397
SVM poly kernel:
accuracy_score:  0.44623655913978494
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.6612903225806451
balanced accuracy score 0.2900924084688268
decission tree depth= 20
accuracy_score:  0.6989247311827957
balanced accuracy score 0.3720903029594105
random forest:
accuracy_score:  0.7849462365591398
balanced accuracy score 0.5835770265528133

"""

    

{3: 359, 4: 8749, 5: 21, 6: 2443, 7: 5311, 8: 2604, 10: 1606, 11: 1, 12: 228, 14: 150, 15: 103, 16: 439, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3091, 24: 8613}
comparing coniferales (Nadelhölzer) to  Malphgiales
labels train set {0.0: 279, 1.0: 259}
labels test set {0.0: 80, 1.0: 100}
SVM linear Kernel:
accuracy_score:  0.5166666666666667
balanced accuracy score 0.050000000000000044
SVM rbf kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
SVM poly kernel:
accuracy_score:  0.4444444444444444
balanced accuracy score 0.0
decission Tree
accuracy_score:  0.7
balanced accuracy score 0.3999999999999999
decission tree depth= 20
accuracy_score:  0.65
balanced accuracy score 0.29000000000000004
random forest:
accuracy_score:  0.7444444444444445
balanced accuracy score 0.51


'\n{3: 372, 4: 8858, 5: 21, 6: 2465, 7: 5668, 8: 2681, 10: 1674, 11: 1, 12: 230, 14: 150, 15: 103, 16: 456, 17: 1, 18: 7, 19: 1, 21: 6, 23: 3100, 24: 8782}\ncomparing coniferales (Nadelhölzer) to  Malphgiales\nlabels train set {0.0: 289, 1.0: 269}\nlabels test set {0.0: 83, 1.0: 103}\nSVM linear Kernel:\naccuracy_score:  0.5483870967741935\nbalanced accuracy score 0.10258509767224222\nSVM rbf kernel:\naccuracy_score:  0.46236559139784944\nbalanced accuracy score 0.026786758685226397\nSVM poly kernel:\naccuracy_score:  0.44623655913978494\nbalanced accuracy score 0.0\ndecission Tree\naccuracy_score:  0.6612903225806451\nbalanced accuracy score 0.2900924084688268\ndecission tree depth= 20\naccuracy_score:  0.6989247311827957\nbalanced accuracy score 0.3720903029594105\nrandom forest:\naccuracy_score:  0.7849462365591398\nbalanced accuracy score 0.5835770265528133\n\n'

In [14]:
"""jetzt nochmal mit up- und dwonsampling !!!
Dann mit Farbe!
Dann Convex Hull?
"""
 

'jetzt nochmal mit up- und dwonsampling !!!\nDann mit Farbe!\nDann Convex Hull?\n'

In [15]:
"""Klasifizierung nach Klasse
unbalanciertes Set
Ohne Farben
   
"""

#set labels to class-labels
labels = np.array(list(map(get_class, labels_unedited)))

#mappimg ginko as conifera
labels = np.array(list(map(lambda x: 2 if x==3 else x, labels)))

labels, _ =scale_y(labels)

print(len(labels), " individuums in train/test-set")

#count_labels 
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

labels = np.array(labels)
features = np.array(features)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)
l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))
train(X_train, X_test, y_train, y_test)



"""
Outout:
    34576  individuums in train/test-set
{0: 34054, 1: 522}
labels train set {0: 22818, 1: 347}
labels test set {0: 11236, 1: 175}
SVM linear Kernel:
accuracy_score:  0.5669967575146788
balanced accuracy score 0.16648273406906355
SVM rbf kernel:
accuracy_score:  0.9199894838313908
balanced accuracy score 0.05807455627320346
SVM spoly kernel:
accuracy_score:  0.017001139251599334
balanced accuracy score -0.00955957890454151
decission Tree
accuracy_score:  0.9705547278941372
balanced accuracy score 0.053174490159182186
decission tree depth= 20
accuracy_score:  0.9426868810796599
balanced accuracy score 0.09800132228042524
random forest:
accuracy_score:  0.9846639207781964
balanced accuracy score 0.0   
    
"""

33733  individuums in train/test-set
{0: 33224, 1: 509}


ValueError: Found input variables with inconsistent numbers of samples: [718, 33733]

In [ ]:

"""Klasifizierung nach Ordung
unbalanciertes Set
Ohne Farben   
"""
features, labels_unedited =get_data()

labels = np.array(list(map(get_order, labels_unedited)))

#mappimg ginko as conifera
print(len(labels), " individuums in train/test-set")

labels, _ =scale_y(labels)
#count_labels 
l, l_num = np.unique(labels, return_counts=True)
print(dict(zip(l, l_num)))

labels = np.array(labels)
features = np.array(features)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)
l, l_num = np.unique(y_train, return_counts=True)
print("labels train set", dict(zip(l, l_num)))
l, l_num = np.unique(y_test, return_counts=True)
print("labels test set", dict(zip(l, l_num)))

train(X_train, X_test, y_train, y_test)


In [ ]:
#importing point cloud:

csv_folder ='clusters/dbscan'
extension = '*.csv' 
features_list = []
for file in tqdm(Path(csv_folder).glob(extension)):
        df = pd.read_csv(file)
        features_list.append(df)
print(len(features_list), " files imported")
individuals = []
for df in tqdm(features_list):     
        cluster_ids = np.unique(np.array(df['Cluster_ID']).astype(int))
        for id in cluster_ids:
                df_split =df[df['Cluster_ID'] == id]
                individuals.append(df_split)
print(len(individuals), "trees imported")
